In [1]:
import pyspark
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession

# from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("ML_Model").getOrCreate()
# spark.conf.set("spark.executor.memory", "8g")
# spark.conf.set("spark.executor.cores", "4")
# spark.conf.set("spark.driver.cores", "4")
# conf = SparkConf().set("spark.jars", "/Users/sz904/Downloads/postgresql-42.2.18.jar")
# sc = SparkContext( conf=conf)



22/11/28 19:44:17 WARN Utils: Your hostname, STEVENs-MacBook-Pro-4.local resolves to a loopback address: 127.0.0.1; using 172.25.2.158 instead (on interface en0)
22/11/28 19:44:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/28 19:44:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/28 19:44:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/28 19:44:18 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [9]:

train_path = "train70_reduced.csv"
test_path = "test30_reduced.csv"


In [10]:
from pyspark import SparkFiles
from pyspark.sql.functions import lit


col_names = [
    "tcp.flags",
    "tcp.time_delta",
    "tcp.len",
    "mqtt.conack.flags",
    "mqtt.conack.flags.reserved",
    "mqtt.conack.flags.sp",
    "mqtt.conack.val",
    "mqtt.conflag.cleansess",
    "mqtt.conflag.passwd",
    "mqtt.conflag.qos",
    "mqtt.conflag.reserved",
    "mqtt.conflag.retain",
    "mqtt.conflag.uname",
    "mqtt.conflag.willflag",
    "mqtt.conflags",
    "mqtt.dupflag",
    "mqtt.hdrflags",
    "mqtt.kalive",
    "mqtt.len",
    "mqtt.msg",
    "mqtt.msgid",
    "mqtt.msgtype",
    "mqtt.proto_len",
    "mqtt.protoname",
    "mqtt.qos",
    "mqtt.retain",
    "mqtt.sub.qos",
    "mqtt.suback.qos",
    "mqtt.ver",
    "mqtt.willmsg",
    "mqtt.willmsg_len",
    "mqtt.willtopic",
    "mqtt.willtopic_len",
    "target"
]



df_train = spark.read.csv(train_path, header=True, inferSchema= True).toDF(*col_names)
df_test = spark.read.csv(test_path, header=True, inferSchema= True).toDF(*col_names)

In [11]:
df_train = df_train.withColumn("dataset", lit(0))
df_test = df_test.withColumn("dataset", lit(1))

df_train.show(1, vertical=True)
df_test.show(1, vertical=True)

print(df_train.count())
print(df_test.count())

22/11/28 19:47:27 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
-RECORD 0--------------------------------
 tcp.flags                  | 0x00000018 
 tcp.time_delta             | 0.998867   
 tcp.len                    | 10         
 mqtt.conack.flags          | 0          
 mqtt.conack.flags.reserved | 0.0        
 mqtt.conack.flags.sp       | 0.0        
 mqtt.conack.val            | 0.0        
 mqtt.conflag.cleansess     | 0.0        
 mqtt.conflag.passwd        | 0.0        
 mqtt.conflag.qos           | 0.0        
 mqtt.conflag.reserved      | 0.0        
 mqtt.conflag.retain        | 0.0        
 mqtt.conflag.uname         | 0.0        
 mqtt.conflag.willflag      | 0.0        
 mqtt.conflags              | 0          
 mqtt.dupflag               | 0.0        
 mqtt.hdrflags              | 0x00000030 
 mqtt.kalive                | 0.0        
 mqtt.len             

In [12]:
df_train.printSchema()
df_test.printSchema()

root
 |-- tcp.flags: string (nullable = true)
 |-- tcp.time_delta: double (nullable = true)
 |-- tcp.len: integer (nullable = true)
 |-- mqtt.conack.flags: string (nullable = true)
 |-- mqtt.conack.flags.reserved: double (nullable = true)
 |-- mqtt.conack.flags.sp: double (nullable = true)
 |-- mqtt.conack.val: double (nullable = true)
 |-- mqtt.conflag.cleansess: double (nullable = true)
 |-- mqtt.conflag.passwd: double (nullable = true)
 |-- mqtt.conflag.qos: double (nullable = true)
 |-- mqtt.conflag.reserved: double (nullable = true)
 |-- mqtt.conflag.retain: double (nullable = true)
 |-- mqtt.conflag.uname: double (nullable = true)
 |-- mqtt.conflag.willflag: double (nullable = true)
 |-- mqtt.conflags: string (nullable = true)
 |-- mqtt.dupflag: double (nullable = true)
 |-- mqtt.hdrflags: string (nullable = true)
 |-- mqtt.kalive: double (nullable = true)
 |-- mqtt.len: double (nullable = true)
 |-- mqtt.msg: string (nullable = true)
 |-- mqtt.msgid: double (nullable = true)
 |-

In [13]:
new_cols = [
    '_'.join(item.split('.')) for item in col_names[:-1]
]

renamed_columns_df = df_train.union(df_test)    # concatenate train & test
for i in range(len(new_cols)):
    renamed_columns_df = renamed_columns_df.withColumnRenamed(col_names[i], new_cols[i])
    df_train = df_train.withColumnRenamed(col_names[i], new_cols[i])
    df_test = df_test.withColumnRenamed(col_names[i], new_cols[i])

print(renamed_columns_df.count())
renamed_columns_df.printSchema()

330936
root
 |-- tcp_flags: string (nullable = true)
 |-- tcp_time_delta: double (nullable = true)
 |-- tcp_len: integer (nullable = true)
 |-- mqtt_conack_flags: string (nullable = true)
 |-- mqtt_conack_flags_reserved: double (nullable = true)
 |-- mqtt_conack_flags_sp: double (nullable = true)
 |-- mqtt_conack_val: double (nullable = true)
 |-- mqtt_conflag_cleansess: double (nullable = true)
 |-- mqtt_conflag_passwd: double (nullable = true)
 |-- mqtt_conflag_qos: double (nullable = true)
 |-- mqtt_conflag_reserved: double (nullable = true)
 |-- mqtt_conflag_retain: double (nullable = true)
 |-- mqtt_conflag_uname: double (nullable = true)
 |-- mqtt_conflag_willflag: double (nullable = true)
 |-- mqtt_conflags: string (nullable = true)
 |-- mqtt_dupflag: double (nullable = true)
 |-- mqtt_hdrflags: string (nullable = true)
 |-- mqtt_kalive: double (nullable = true)
 |-- mqtt_len: double (nullable = true)
 |-- mqtt_msg: string (nullable = true)
 |-- mqtt_msgid: double (nullable = tr

In [14]:
import pyspark.sql.functions as F

casted_types_df = renamed_columns_df
casted_types_df.select("tcp_flags").distinct().show(vertical=True)
casted_types_df.select("mqtt_conack_flags").distinct().show(vertical=True)
casted_types_df.select("mqtt_conflags").distinct().show(vertical=True)
casted_types_df.select("mqtt_hdrflags").distinct().show(vertical=True)
casted_types_df.select("mqtt_msg").distinct().show(vertical=True)
casted_types_df.select("mqtt_protoname").distinct().show(vertical=True)
casted_types_df.select("mqtt_protoname").groupBy("mqtt_protoname").count().show()
casted_types_df.select("target").show(vertical=True)

# casted_types_df = renamed_columns_df.withColumn(
#     "new_tcp_flags", 
#     F.expr("conv(substring(tcp_flags, 3), 16, 10)")
# ).drop("tcp_flags").distinct()      # deleting duplicate rows

-RECORD 0---------------
 tcp_flags | 0x00000004 
-RECORD 1---------------
 tcp_flags | 0x00000010 
-RECORD 2---------------
 tcp_flags | 0x00000019 
-RECORD 3---------------
 tcp_flags | 0x00000002 
-RECORD 4---------------
 tcp_flags | 0x00000012 
-RECORD 5---------------
 tcp_flags | 0x00000014 
-RECORD 6---------------
 tcp_flags | 0x00000018 
-RECORD 7---------------
 tcp_flags | 0x00000011 

-RECORD 0-----------------------
 mqtt_conack_flags | 0          
-RECORD 1-----------------------
 mqtt_conack_flags | 0x00000000 

-RECORD 0-------------------
 mqtt_conflags | 0          
-RECORD 1-------------------
 mqtt_conflags | 0x00000002 
-RECORD 2-------------------
 mqtt_conflags | 0x000000c2 
-RECORD 3-------------------
 mqtt_conflags | 0x00000082 

-RECORD 0-------------------
 mqtt_hdrflags | 0x00000031 
-RECORD 1-------------------
 mqtt_hdrflags | 0x00000090 
-RECORD 2-------------------
 mqtt_hdrflags | 0          
-RECORD 3-------------------
 mqtt_hdrflags | 0x000000d0 
-

In [15]:
from pyspark.ml import Pipeline,Transformer
from pyspark.ml.feature import Imputer,StandardScaler,StringIndexer,OneHotEncoder, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np

In [16]:
df_train_pandas = df_train.toPandas()
df_test_pandas = df_test.toPandas()


In [17]:
###### Converting tcp_flags, mqtt_conflags, mqtt_hdrflags
###### from hex string to int (decimal)


tcp_flags = list(df_train.select("tcp_flags").toPandas()["tcp_flags"])
mqtt_conflags = list(df_train.select("mqtt_conflags").toPandas()["mqtt_conflags"])
mqtt_hdrflags = list(df_train.select("mqtt_hdrflags").toPandas()["mqtt_hdrflags"])

tcp_flags_test = list(df_test.select("tcp_flags").toPandas()["tcp_flags"])
mqtt_conflags_test = list(df_test.select("mqtt_conflags").toPandas()["mqtt_conflags"])
mqtt_hdrflags_test = list(df_test.select("mqtt_hdrflags").toPandas()["mqtt_hdrflags"])


tcp_flags_decimal = []
mqtt_conflags_decimal = []
mqtt_hdrflags_decimal = []

tcp_flags_decimal_test = []
mqtt_conflags_decimal_test = []
mqtt_hdrflags_decimal_test = []

for i in range(len(tcp_flags)):
    tcp_flags_decimal.append(int(tcp_flags[i], 16)) 
    mqtt_conflags_decimal.append(int(mqtt_conflags[i], 16)) 
    mqtt_hdrflags_decimal.append(int(mqtt_hdrflags[i], 16)) 
    
for i in range(len(tcp_flags_test)):
    tcp_flags_decimal_test.append(int(tcp_flags_test[i], 16)) 
    mqtt_conflags_decimal_test.append(int(mqtt_conflags_test[i], 16)) 
    mqtt_hdrflags_decimal_test.append(int(mqtt_hdrflags_test[i], 16)) 
    
df_train_pandas["tcp_flags_decimal"] = tcp_flags_decimal
df_train_pandas["mqtt_conflags_decimal"] = mqtt_conflags_decimal
df_train_pandas["mqtt_hdrflags_decimal"] = mqtt_hdrflags_decimal

df_test_pandas["tcp_flags_decimal"] = tcp_flags_decimal_test
df_test_pandas["mqtt_conflags_decimal"] = mqtt_conflags_decimal_test
df_test_pandas["mqtt_hdrflags_decimal"] = mqtt_hdrflags_decimal_test
# df_train.drop("tcp_flags")  



In [18]:
import time
start = time.time()


df_test_pandas.to_csv( "df_test_pandas.csv", index=False)
df_train_pandas.to_csv( "df_train_pandas.csv", index=False)

df_train_updated=spark.createDataFrame(df_train_pandas) 
df_test_updated=spark.createDataFrame(df_test_pandas) 


print(time.time() - start)

/Users/sz904/miniforge3/envs/14813project/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/Users/sz904/miniforge3/envs/14813project/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


37.931740045547485
